# Power System Planning Project

## 1. Generation Expansion and Long-Term Storage Planning

### 1.1. Mathematical Formulation

$\begin{align}
\min \quad &\sum_{g\in G} C^{G,F} CAP_g + \sum_{r\in R} C^{R,F} CAP_r + \sum_{s\in S} (C^{S,P} CAP_s + C^{S,E} SOCmax_s) + \\
&\sum_{t\in T} \left(\sum_{g\in G} C^{G,V} GEN_{g,t} + C^{G,S} START_{g,t} + \sum_{r\in R} C^{R,V} REN_{r,t}+VoLL\times NSE_t\right)\\
\text{s.t. } \quad &\sum_{g\in G} GEN_{g,t} + \sum_{r\in R} REN_{r,t} + \sum_{s\in S} \left(DISCHARGE_{s,t} - CHARGE_{s,t} \right) + NSE_t = L_t &\forall t\in T \\
&0 \leq GEN_{g,t} \leq CAP_g \times COMMIT_{g,t} &\forall g\in G, \forall t\in T \\
&COMMIT_{g,t} >= \sum^{t}_{t'\geq t - MinUp_g} START_{g,t} &\forall g\in G, \forall t\in T \\
&1 - COMMIT_{g,t} >= \sum^{t}_{t'\geq t - MinDown_g} SHUT_{g,t} &\forall g\in G, \forall t\in T \\
&COMMIT_{g,t+1} - COMMIT_{g,t} = START_{g,t+1} - SHUT_{g,t+1} &\forall g\in G, t = 1..T-1\\
&GEN_{g,t+1} - GEN_{g,t} \leq RampUp_g &\forall g\in G, t = 1..T-1 \\
&GEN_{g,t} - GEN_{g,t+1} \leq RampDown_g &\forall g\in G, t = 1..T-1 \\
&0 \leq REN_{r,t}    \leq CAP_r \times cf_{r,t} &\forall r\in R, \forall t\in T \\
&REN_{r,t+1} - REN_{r,t} \leq RampUp_r &\forall r\in R, t = 1..T-1 \\
&REN_{r,t} - REN_{r,t+1} \leq RampDown_r &\forall r\in R, t = 1..T-1 \\
&0 \leq CHARGE_{s,t} \leq CAP_s &\forall s\in S, \forall t\in T \\
&0 \leq DISCHARGE_{s,t} \leq CAP_s &\forall s\in S, \forall t\in T \\
&0 \leq SOC_{s,t} \leq SOCmax_s &\forall s\in S, \forall t\in T \\
&SOC_{s,t+1} = SOC_{s,t} + CHARGE_{s,t+1} \times Eff_s - DISCHARGE_{s,t+1}/Eff_s &\forall s\in S, t = 1..T-1 \\
&SOC_{s,min(T)} = SOC_{s,max(T)} = 0.5 \times SOCmax_s &\forall s\in S \\
&NSE_t \geq 0 &\forall t\in T \\
\end{align}$

### 1.2. Modelling with JuMP

In [51]:
using JuMP, HiGHS
using DataFrames, CSV, XLSX

stor_info = DataFrame(CSV.File("data/sample/stor_info.csv"))
ther_info = DataFrame(CSV.File("data/sample/ther_info.csv"))
ren_info = DataFrame(CSV.File("data/sample/ren_info.csv"))
load_forecast = DataFrame(CSV.File("data/sample/load_forecast.csv"))
capacity_factor = DataFrame(CSV.File("data/sample/capacity_factor.csv"))

stor_info = stor_info[:,[:id, :name, :power_cost, :energy_cost, :eff]]
ther_info = ther_info[:,[:id, :name, :fixed_cost, :var_cost, :start_up_cost, :ramp_up_rate, :ramp_dn_rate, :min_up_time, :min_dn_time]]
ren_info = ren_info[:,[:id, :name, :fixed_cost, :var_cost, :ramp_up_rate, :ramp_dn_rate]]
load_forecast = load_forecast[:,[:hour, :load]]
capacity_factor = capacity_factor[:,[:hour, :r_id, :cf]];


In [52]:
# stor_info = id, name, power_cost, energy_cost, eff
# ther_info = id, name, fixed_cost, var_cost, start_up_cost, ramp_up_rate, ramp_dn_rate, min_up_time, min_dn_time
# ren_info = id, name, fixed_cost, var_cost, ramp_up_rate, ramp_dn_rate
# load_forecast = hour, load
# capacity_factor = hour, r_id, cf

function GESP(stor_info, ther_info, ren_info, load_forecast, capacity_factor, VoLL=30000, bigM = 10000, mip_gap=0.01)

    # SETS
    G = ther_info.id
    R = ren_info.id
    S = stor_info.id
    T = load_forecast.hour
    T1 = T[2:end]

    # INIT
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)
    set_optimizer_attribute(model, "mip_rel_gap", mip_gap)

    # VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        CAPG_COMMIT[G,T] >= 0
        CAPR[R] >= 0
        CAPS[S] >= 0
        SOCM[S] >= 0
        GEN[G,T] >= 0
        REN[R,T] >= 0
        CHARGE[S,T] >= 0
        DISCHARGE[S,T] >= 0
        SOC[S,T] >= 0
        COMMIT[G,T], Bin
        START[G,T], Bin
        SHUT[G,T], Bin
        NSE[T] >= 0
    end)

    # INVESTMENT COSTS
    @expression(model, ThermalIC, sum(ther_info[ther_info.id .== g, :fixed_cost][1] * CAPG[g] for g in G))
    @expression(model, RenewableIC, sum(ren_info[ren_info.id .== r, :fixed_cost][1] * CAPR[r] for r in R))
    @expression(model, StoragePowIC, sum(stor_info[stor_info.id .== s, :power_cost][1] * CAPS[s] for s in S))
    @expression(model, StorageEnIC, sum(stor_info[stor_info.id .== s, :energy_cost][1] * SOCM[s] for s in S))
    @expression(model, TIC, ThermalIC + RenewableIC + StoragePowIC + StorageEnIC)

    # OPERATION COSTS
    @expression(model, ThermalOC, sum(ther_info[ther_info.id .== g, :var_cost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, RenewableOC, sum(ren_info[ren_info.id .== r, :var_cost][1] * REN[r,t] for r in R for t in T))
    @expression(model, StartupOC, sum(ther_info[ther_info.id .== g, :start_up_cost][1] * START[g,t] for g in G for t in T))
    @expression(model, NSE_C, sum(VoLL * NSE[t] for t in T))
    @expression(model, TOC, ThermalOC + RenewableOC + StartupOC + NSE_C)

    # OBJECTIVE FUNCTION
    @objective(model, Min, TIC + TOC)

    # POWER BALANCE CONSTRAINT
    @constraint(model, PowerBalanceC[t in T], sum(GEN[g,t] for g in G) + sum(REN[r,t] for r in R) + sum(DISCHARGE[s,t] for s in S) - sum(CHARGE[s,t] for s in S) + NSE[t] == load_forecast[load_forecast.hour.==t,:load][1])

    # THERMAL GENERATION LIMIT CONSTRAINT
    @constraint(model, LessThanMC[g in G, t in T], CAPG_COMMIT[g,t] <= bigM * COMMIT[g,t])
    @constraint(model, LessThanCapC[g in G, t in T], CAPG_COMMIT[g,t] <= CAPG[g])
    @constraint(model, AuxIsRealC[g in G, t in T], CAPG_COMMIT[g,t] >= CAPG[g] - (1- COMMIT[g,t]) * bigM)
    @constraint(model, ThermalLimC[g in G, t in T], GEN[g,t] <= CAPG_COMMIT[g,t])

    # START CONSTRAINT
    @constraint(model, StartC[g in G, t in T], 
        COMMIT[g,t] >= sum(START[g,i] for i in max(1,t-ther_info[ther_info.id.==g,:min_up_time][1]):t))

    # SHUT CONSTRAINT
    @constraint(model, ShutC[g in G, t in T], 
        1 - COMMIT[g,t] >= sum(SHUT[g,i] for i in max(1,t-ther_info[ther_info.id.==g,:min_dn_time][1]):t))

    # COMMIT CONSTRAINT
    @constraint(model, CommitC[g in G, t in T1], COMMIT[g,t] - COMMIT[g,t-1] == START[g,t] - SHUT[g,t])

    # THERMAL RAMP UP CONSTRAINT
    @constraint(model, TherRampUpC[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= ther_info[ther_info.id.==g,:ramp_up_rate][1] * CAPG[g])
    
    # THERMAL RAMP DOWN CONSTRAINT
    @constraint(model, TherRampDnC[g in G, t in T1], 
        GEN[g,t-1] - GEN[g,t] <= ther_info[ther_info.id.==g,:ramp_dn_rate][1] * CAPG[g])

    # VRE LIMIT CONSTRAINT
    @constraint(model, VreC[r in R, t in T], 
        # REN[r,t] <= CAPR[r] * capacity_factor[(capacity_factor.r_id.==r) .& (capacity_factor.hour.==t), :cf][1])
        REN[r,t] <= 500 * capacity_factor[(capacity_factor.r_id.==r) .& (capacity_factor.hour.==t), :cf][1])
    
    # VRE RAMP UP CONSTRAINT
    @constraint(model, VreRampUpC[r in R, t in T1], 
        REN[r,t] - REN[r,t-1] <= ren_info[ren_info.id.==r,:ramp_up_rate][1] * CAPR[r])
    
    # VRE RAMP DOWN CONSTRAINT
    @constraint(model, VreRampDnC[r in R, t in T1], 
        REN[r,t-1] - REN[r,t] <= ren_info[ren_info.id.==r,:ramp_dn_rate][1] * CAPR[r])
    
    # CHARGE LIMIT CONSTRAINT
    @constraint(model, ChargeC[s in S, t in T], CHARGE[s,t] <= CAPS[s])

    # DISCHARGE LIMIT CONSTRAINT
    @constraint(model, DischargeC[s in S, t in T], DISCHARGE[s,t] <= CAPS[s])

    # SOC LIMIT CONSTRAINT
    @constraint(model, SocLimitC[s in S, t in T], SOC[s,t] <= SOCM[s])

    # SOC UPDATE CONSTRAINT
    @constraint(model, SocUpdateC[s in S, t in T1],
        SOC[s, t] == SOC[s,t-1] + (CHARGE[s,t] * stor_info[stor_info.id.==s,:eff][1]) - 
            (DISCHARGE[s,t] / stor_info[stor_info.id.==s,:eff][1]))
    
    # SOC INI CONSTRAINT
    @constraint(model, SocIniC[s in S], SOC[s,1] == 0.5 * CAPS[s])

    # SOC END CONSTRAINT
    @constraint(model, SocEndC[s in S], SOC[s,length(T)] == 0.5 * CAPS[s])

    # SOLVE GESP
    optimize!(model)

    return(
        capg = value.(CAPG).data, 
        capr = value.(CAPR).data, 
        caps = value.(CAPS).data,
        gen = value.(GEN).data,
        ren = value.(REN).data,
        socm = value.(SOCM).data
    )

end
    



GESP (generic function with 4 methods)

In [53]:
sol = GESP(stor_info, ther_info, ren_info, load_forecast, capacity_factor,30000, 10000, 0.01)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
316 rows, 237 cols, 908 nonzeros
296 rows, 231 cols, 890 nonzeros

Solving MIP model with:
   296 rows
   231 cols (24 binary, 0 integer, 0 implied int., 207 continuous)
   890 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   675541.317719   695241.587097      2.83%        0      0      0       162     0.0s
 L       0       0         0   0.00%   675541.317719   675541.317719      0.00%       10      1      0       166     0.0s
 T       0       0         0   0.00%   675541.317719   675541.317719      0.00%       10      1      0       310   

(capg = [-0.0, -0.0, 289.5748800887451], capr = [0.0, 412.8650565], caps = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1487.3812127799424, 0.0, 0.0, 0.0], gen = [-0.0 -0.0 -0.0 -0.0; -0.0 -0.0 -0.0 -0.0; 0.0 57.914976017749 173.74492805324704 289.5748800887451], ren = [0.0 0.0 -0.0 0.0; 0.0 0.0 206.43252825 412.8650565], socm = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 743.6906063899712, 0.0, 0.0, 0.0])

In [54]:
sol.capg

3-element Vector{Float64}:
  -0.0
  -0.0
 289.5748800887451

In [55]:
sol.capr

2-element Vector{Float64}:
   0.0
 412.8650565

In [56]:
sol.caps

14-element Vector{Float64}:
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
 1487.3812127799424
    0.0
    0.0
    0.0

In [57]:
sol.gen

3×4 Matrix{Float64}:
 -0.0  -0.0     -0.0     -0.0
 -0.0  -0.0     -0.0     -0.0
  0.0  57.915  173.745  289.575

In [58]:
sol.ren

2×4 Matrix{Float64}:
 0.0  0.0   -0.0      0.0
 0.0  0.0  206.433  412.865

In [59]:
sol.socm

14-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
 743.6906063899712
   0.0
   0.0
   0.0

## 2. Sensitivity Analysis of Storage Efficiencies

$\begin{align}